# ***Load data from url and convert it to data form***

In [ ]:
# A government database of vehicles
url = 'https://data.gov.il/api/3/action/datastore_search?resource_id=053cea08-09bc-40ec-8f7a-156f0677aff3'

import requests
import pandas as pd

response = requests.get(url)
data = response.json()
df = pd.DataFrame(data)
data_df = pd.DataFrame(data['result']['records'])

# ***Creating the From***

In [ ]:
from ctypes import alignment
import ipywidgets as widgets
from IPython.display import display

# Create a dropdown for selecting a Tozeret (vehicle origin/manufacturer)
tozeret_dropdown = widgets.Dropdown(
    options=data_df['tozeret_nm'].unique(),  # 'tozeret_nm' contains manufacturer names
    description='Tozeret:',
    disabled=False
)

# Create a dropdown for selecting a Kinuy Mishari of certain Tozeret(manufacturer)
kinuy_mishari_dropdown = widgets.Dropdown(
    options=[],  # Initially empty
    description='Kinuy Mishari:',
    style={'description_width': 'initial'},
    disabled=True  # Initially disabled until a Tozeret is selected,
)

# Function to update Kinuy Mishari dropdown options based on Tozeret
def update_kinuy_mishari(selected_tozeret):
    kinuy_mishari_dropdown.options = data_df[data_df['tozeret_nm'] == selected_tozeret]['kinuy_mishari'].unique()
    # Re-enable kinuy_mishari_dropdown after updating
    kinuy_mishari_dropdown.disabled = False

# Function triggered when Tozeret dropdown changes
def on_tozeret_click(change):
    selected_tozeret = change['new']
    if selected_tozeret:
        update_kinuy_mishari(selected_tozeret)  # Update Kinuy Mishari dropdown based on Tozeret
    else:
        print("Error -> Tozeret not found")

# Function triggered when Kinuy Mishari dropdown changes
def on_kinuy_mishari_click(change):
    selected_kinuy = change['new']
    current_tozeret = tozeret_dropdown.value
    if selected_kinuy:
        num_vehicles, unique_ramat_gimur = parse_vehicle_info(current_tozeret, selected_kinuy)
        display_vehicle_info(num_vehicles, unique_ramat_gimur)
    else:
        print("Error -> Kinuy Mishari not found")

def parse_vehicle_info(selected_tozeret, selected_kinuy):
    filtered_df = data_df[(data_df['tozeret_nm'] == selected_tozeret) & (data_df['kinuy_mishari'] == selected_kinuy)]
    num_vehicles = len(filtered_df)
    unique_ramat_gimur = filtered_df['ramat_gimur'].unique()
    return num_vehicles, unique_ramat_gimur


# Function to display vehicle info
def display_vehicle_info(num_vehicles, unique_ramat_gimur):
    print(f"Total records: {num_vehicles}")
    print(f"Unique Ramat Gimur: {', '.join(unique_ramat_gimur)}")

# Link the Tozeret dropdown selection to the on_tozeret_click function
tozeret_dropdown.observe(on_tozeret_click, names='value')

# Link the Kinuy Mishari dropdown selection to the on_kinuy_mishari_click function
kinuy_mishari_dropdown.observe(on_kinuy_mishari_click, names='value')

# ***Creating tab content***

In [ ]:
import plotly.graph_objects as go
import matplotlib.pyplot as plt
import seaborn as sns

# Tab 1: Data Overview
tab1_content = widgets.Output()
with tab1_content:
    print("Data Overview:")
    display(data_df.describe())  # Summary statistics

# Tab 2: Raw Data
tab2_content = widgets.Output()
with tab2_content:
    print("Raw Data:")
    display(data_df)  # Full DataFrame

# Tab 3: number of cars created per year
tab3_content = widgets.Output()
with tab3_content:
  gimur_counts = data_df['shnat_yitzur'].value_counts()

  plt.figure(figsize=(8, 6))
  sns.barplot(x=gimur_counts.index, y=gimur_counts.values, color='purple')
  plt.title('Count of cars by Year')
  plt.xlabel('Year')
  plt.ylabel('Count')
  plt.xticks(rotation=45)
  plt.show()

# Tab 4: dropdowns
tab4_content = widgets.Output()
with tab4_content:
  print("Dropdowns")
  # Display the dropdowns
  display(tozeret_dropdown)
  display(kinuy_mishari_dropdown)

# ***Presenting Tabs***

In [ ]:
# Create Tabs
tabs = widgets.Tab(children=[tab1_content, tab2_content, tab3_content, tab4_content])
tabs.set_title(0, 'Tab 1: Data Overview')
tabs.set_title(1, 'Tab 2: Raw Data')
tabs.set_title(2, 'Tab 3: Charts')
tabs.set_title(3, 'Tab 4: Dropdowns')



# Display Tabs
display(tabs)

Total records: 1420
Unique Ramat Gimur: ACTIVE, COMFORT, SPIRIT
Total records: 88
Unique Ramat Gimur: LX, GLX
